In [1]:
print("BEGIN train smart-filter_spam")

In [1]:
assert(corpus_bow)

In [1]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score
import pickle as c
import numpy as np
import time                       # performance/completion feedback

In [1]:
def right(s, amount):
    return s[-amount:]

def pad(s, pad_char='0', pad_sz=2):
    return right(pad_char*pad_sz + s, pad_sz)
   

In [1]:
# LOCAL OVERRIDES FOR DEV + TEST

#USERARG_TRAINvTEST_SPLIT    = 0.2
#USERARG_SPAM_CLASSIFIER     = {'NAIVEBAYES', 'ADABOOST'}

def spam_train_filename():
    t = time.localtime()
    t = str(t.tm_year) + pad(str(t.tm_mon)) + pad(str(t.tm_mday)) + pad(str(t.tm_hour)) + str(int(t.tm_min/20)*20)
    SPAMNB_USERARG_PATH  ='models/spam__arg-pos=' + str(USERARG_FILTER_POS) 
    SPAMNB_USERARG_PATH +='-dblim=' + str(USERARG_RECORD_LIMITER) 
    SPAMNB_USERARG_PATH +='-lim=' + str(USERARG_FILTER_WORDS) 
    SPAMNB_USERARG_PATH +='-tts=' + str(USERARG_TRAINvTEST_SPLIT) 
    SPAMNB_USERARG_PATH +='-acc=' + str(_accuracy) 
    SPAMNB_USERARG_PATH +='-cfy=' + USERARG_SPAM_CLASSIFIER
    SPAMNB_USERARG_PATH += '-db=' + USERARG_DATABASEFILE
    SPAMNB_USERARG_PATH += '-dt=' + t
    SPAMNB_USERARG_PATH += '-r#=' + str(_RUN_INTERATION)
    SPAMNB_USERARG_PATH +='.model'
    t = None
    return SPAMNB_USERARG_PATH

#%run ./common_preprocessor.ipynb

In [1]:
# intermediate VARIABLES
_labels = []
iTime = 0      # used to track execution performance

In [1]:
print("\tExtracting features and supervision-labels from database...")
iTime = time.time()

_labels = list(database[i][FIELD_BzNDX_BIN_CLASS] for i in range(len(database))) # {FIELD_BzNDX_BIN_CLASS, FIELD_BzNDX_INT_CLASS}
print("\tINFO\TYPE: features dimensions:",len(corpus_bow), "x", len(corpus_bow[0]))
print("\tINFO\TYPE: features[i]", type(corpus_bow[0])," example:",corpus_bow[0])

print("\tCompleted in", round(time.time() - iTime,3),"seconds")

In [1]:
print("\tTraining Naive Bayes Filter...")
print("\tNOTE: performance depends on randomly selected training/test data split...")
iTime = time.time()

# instantiate classifier
if USERARG_SPAM_CLASSIFIER == 'NAIVEBAYES':
    model_spam = MultinomialNB()
elif USERARG_SPAM_CLASSIFIER == 'ADABOOST':
    model_spam = AdaBoostClassifier()

# shuffle and partition dataset
x_train,x_test, y_train,y_test = tts(corpus_bow, _labels, test_size=USERARG_TRAINvTEST_SPLIT)
_labels   = None
tts = None

# for the model to be portable across datasets, make sure the training data is a matrix
x_train = np.asarray(x_train)
print("\tASSERT: x_train (", type(x_train), ") is array-like (not a list) = ", type(x_train) != type(list()))
print("\tINFO: x_train dimensions =", len(x_train), "x",len(x_train[0]))
print("\tINFO: x_test  dimensions =", len(x_test),  "x", len(x_test[0]))
print("\tINFO: y_train dimensions =", len(y_train))
print("\tINFO: y_test  dimensions =", len(y_test))

# train classifier
model_spam.fit(x_train, y_train)


# test classifier
_predictions = model_spam.predict(x_test)
_accuracy = round(accuracy_score(y_test,  _predictions),2)
print("\tAchieved",round(_accuracy*100,0), "% accuracy")
_predictions = None
x_train = None
x_test = None
y_train = None
y_test = None

print("\tCompleted in", round(time.time() - iTime,3),"seconds")

In [1]:
# save classifier
with open(spam_train_filename(), 'wb') as fp:
    c.dump(model_spam, fp)


In [1]:
print("END train smart-filter_spam")